# 基礎班時問 1330

問1. 怎麼用python抓share point的資料

程式相關問題通常把問題翻譯成英文的簡短句子丟到 google 裡, 第一筆 stack overflow 的資料有 5, 6 成是解答

https://stackoverflow.com/questions/53671547/python-download-files-from-sharepoint-site

# 影像班時問 1430

In [ ]:
問1.  怎麼使用 GPU 加速 YOLO


- 1. 查 torch 版本
- 2. 安裝相對應的 CUDA toolkit
- 3. 重新安裝 torch 與 cuda 相關的套件

https://developer.nvidia.com/cuda-toolkit-archive/
https://pytorch.org/get-started/previous-versions/

cuda 12.1
pytorch 2.2.2
會使用以下設定
conda install pytorch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2 pytorch-cuda=12.1 -c pytorch -c nvidia


In [3]:
import torch

In [4]:
torch.cuda.is_available()

True

In [2]:
from ultralytics import YOLO

In [8]:
model = YOLO('yolov8n.pt')

100%|█████████████████████████████████████████████████████████████████████████████| 6.23M/6.23M [00:00<00:00, 6.56MB/s]


In [12]:
model = model.to('cuda')

In [15]:
model

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

## yolo 模型使用

In [ ]:
from

## yolo 模型訓練步驟
- 準備資料
- 選擇模型
- 訓練
- 使用模型


# 綜合時間 1530